In [1]:
import os, sys, shutil
import h5py
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
import matplotlib.pyplot as plt
import seaborn as sns
import math
%matplotlib inline
from tqdm import tqdm
from PIL import Image
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [ ]:
python '/Users/Alex/Documents/Cover/car_classifier/image_classifier/retrain.py' --image_dir '/Volumes/alex_encrypted/model_v1/train/' \
--tfhub_module 'https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/3'

cd /Users/Alex/Documents/Cover/car_classifier/image_classifier/

python '/Users/Alex/Documents/Cover/car_classifier/image_classifier/retrain.py' \
--output_graph '/Volumes/alex_encrypted/model_v1/train/retrained_graph.tflite' \
--output_labels '/Volumes/alex_encrypted/model_v1/train/retrained_labels.txt' \
--image_dir '/Volumes/alex_encrypted/model_v1/train/' \
--architecture mobilenet_1.0_224

In [ ]:
# retrain.py arguments:
--architecture mobilenet_1.0_224
--output_graph=tf_files/retrained_graph.pb   \
--output_labels=tf_files/retrained_labels.txt   \
--learning_rate=0.01   \
--architecture=mobilenet_0.50_160   \
--image_dir=../Pokemons-subset \
--flip_left_right \
--random_crop=10 \
--random_scale=10 \
--random_brightness=10

In [ ]:
#convert to tflite file
toco \
--graph_def_file='/Volumes/alex_encrypted/model_mobilenet_v1_1.0_224/output_graph.pb' \
--output_file='/Volumes/alex_encrypted/model_mobilenet_v1_1.0_224/output_graph.tflite' \
--input_format=TENSORFLOW_GRAPHDEF \
--output_format=TFLITE \
--input_shape=1,224,224,3 \
--input_array=input \
--output_array=final_result \
--inference_type=FLOAT \
--input_data_type=FLOAT

In [ ]:
#copy all images from subdirectory into one folder

rootdir = '/Volumes/alex_encrypted/vehicle'
target_dir = '/Volumes/alex_encrypted/labelled/'

file_count = 0

for subdir, dirs, files in os.walk(rootdir):
    file_count += 1
    file_count_str = f'{file_count:06}'
    for file in files:
        #print os.path.join(subdir, file)
        filepath = subdir + os.sep + file

        if filepath.endswith('.jpg'):
            view_name = file[:-4]
            file_out = target_dir + view_name + '/' + file_count_str + '.jpg'
            print(filepath)
            print(file_out)
            
            shutil.copy(filepath, file_out)

In [ ]:
train_path = '/Volumes/alex_encrypted/model_v1/train/'

#'/Users/Alex/Documents/Cover/car_classifier/image_classifier/'

In [2]:
# Create dictionary with input and output file paths
in_path = '/Volumes/alex_encrypted/labelled_v1/'
out_path = '/Volumes/alex_encrypted/model_v1/'

label_dict = {f.name: {'in_path':f.path} for f in os.scandir(in_path) if f.is_dir()}

for label, info in label_dict.items():
    for (_, _, filenames) in os.walk(info['in_path']):
        info['out_path'] = os.path.join(out_path, label)
        info['files'] = filenames
        break

In [3]:
# Split into Test and Train datasets
for label, info in label_dict.items():
    info['train'], info['test'] = train_test_split(info['files'], test_size = 250, train_size = 1000, random_state = 0)

In [4]:
def checkpath(path_name):
    if not os.path.exists(path_name): os.makedirs(path_name)

In [5]:
# Copy data to folders
train_test = ['train', 'test']

for label, info in label_dict.items():
    for func in train_test:
        for file in info[func]:
            from_file = os.path.join(info['in_path'], file) 
            to_file = os.path.join(info['out_path'], func, file)
            os.makedirs(os.path.dirname(from_file), exist_ok=True)
            os.makedirs(os.path.dirname(to_file), exist_ok=True)
            # print(from_file, '  -->   ', to_file)
            shutil.copy(from_file, to_file)
            

In [ ]:
label_dict['car']

In [6]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest') 

for label, info in label_dict.items():
    count1 = 0
    for file in info['train']:
        img_in_path = os.path.join(info['out_path'], 'train', file)
        img_out_path = os.path.join(info['out_path'], 'train2')
        checkpath(img_out_path)
        # for file in os.listdir(src_lab_dir):
        img = load_img(img_in_path)
        #img.save(os.path.join(dest_lab_dir,file))
        x = img_to_array(img) 
        x = x.reshape((1,) + x.shape)
        i = 0
        for batch in datagen.flow(x, batch_size=1,save_to_dir=img_out_path, save_format='jpg'): break
        #datagen.flow(x, batch_size=1,save_to_dir=img_out_path, save_format='jpg')
        count1 +=1
        print(label, ': ', count1, ' ', img_out_path, '/', file)

car :  1   /Volumes/alex_encrypted/model_v1/car/train2 / 000452 4.jpg
car :  2   /Volumes/alex_encrypted/model_v1/car/train2 / 000986 4.jpg
car :  3   /Volumes/alex_encrypted/model_v1/car/train2 / 000622 2.jpg
car :  4   /Volumes/alex_encrypted/model_v1/car/train2 / 000023 3.jpg
car :  5   /Volumes/alex_encrypted/model_v1/car/train2 / 000694 3.jpg
car :  6   /Volumes/alex_encrypted/model_v1/car/train2 / 001092 2.jpg
car :  7   /Volumes/alex_encrypted/model_v1/car/train2 / 000354 4.jpg
car :  8   /Volumes/alex_encrypted/model_v1/car/train2 / 000304 2.jpg
car :  9   /Volumes/alex_encrypted/model_v1/car/train2 / 000609.jpg
car :  10   /Volumes/alex_encrypted/model_v1/car/train2 / 000522 2.jpg
car :  11   /Volumes/alex_encrypted/model_v1/car/train2 / 000942 2.jpg
car :  12   /Volumes/alex_encrypted/model_v1/car/train2 / 001092.jpg
car :  13   /Volumes/alex_encrypted/model_v1/car/train2 / 001300 4.jpg
car :  14   /Volumes/alex_encrypted/model_v1/car/train2 / 000156 4.jpg
car :  15   /Volume

car :  117   /Volumes/alex_encrypted/model_v1/car/train2 / 000186 3.jpg
car :  118   /Volumes/alex_encrypted/model_v1/car/train2 / 000161 4.jpg
car :  119   /Volumes/alex_encrypted/model_v1/car/train2 / 001223.jpg
car :  120   /Volumes/alex_encrypted/model_v1/car/train2 / 000312 3.jpg
car :  121   /Volumes/alex_encrypted/model_v1/car/train2 / 000758 3.jpg
car :  122   /Volumes/alex_encrypted/model_v1/car/train2 / 000395 2.jpg
car :  123   /Volumes/alex_encrypted/model_v1/car/train2 / 000418 2.jpg
car :  124   /Volumes/alex_encrypted/model_v1/car/train2 / 000149 4.jpg
car :  125   /Volumes/alex_encrypted/model_v1/car/train2 / 000062 4.jpg
car :  126   /Volumes/alex_encrypted/model_v1/car/train2 / 001135 3.jpg
car :  127   /Volumes/alex_encrypted/model_v1/car/train2 / 001210 2.jpg
car :  128   /Volumes/alex_encrypted/model_v1/car/train2 / 001157 3.jpg
car :  129   /Volumes/alex_encrypted/model_v1/car/train2 / 000976 4.jpg
car :  130   /Volumes/alex_encrypted/model_v1/car/train2 / 000783 

car :  232   /Volumes/alex_encrypted/model_v1/car/train2 / 000648.jpg
car :  233   /Volumes/alex_encrypted/model_v1/car/train2 / 000620.jpg
car :  234   /Volumes/alex_encrypted/model_v1/car/train2 / 001232 4.jpg
car :  235   /Volumes/alex_encrypted/model_v1/car/train2 / 000284 3.jpg
car :  236   /Volumes/alex_encrypted/model_v1/car/train2 / 000159.jpg
car :  237   /Volumes/alex_encrypted/model_v1/car/train2 / 000422 4.jpg
car :  238   /Volumes/alex_encrypted/model_v1/car/train2 / 000307.jpg
car :  239   /Volumes/alex_encrypted/model_v1/car/train2 / 000076.jpg
car :  240   /Volumes/alex_encrypted/model_v1/car/train2 / 001251 3.jpg
car :  241   /Volumes/alex_encrypted/model_v1/car/train2 / 000430.jpg
car :  242   /Volumes/alex_encrypted/model_v1/car/train2 / 000708 2.jpg
car :  243   /Volumes/alex_encrypted/model_v1/car/train2 / 000776 4.jpg
car :  244   /Volumes/alex_encrypted/model_v1/car/train2 / 001201 2.jpg
car :  245   /Volumes/alex_encrypted/model_v1/car/train2 / 000216 3.jpg
car 

car :  347   /Volumes/alex_encrypted/model_v1/car/train2 / 001200.jpg
car :  348   /Volumes/alex_encrypted/model_v1/car/train2 / 000923 3.jpg
car :  349   /Volumes/alex_encrypted/model_v1/car/train2 / 000047.jpg
car :  350   /Volumes/alex_encrypted/model_v1/car/train2 / 000447 2.jpg
car :  351   /Volumes/alex_encrypted/model_v1/car/train2 / 000016 4.jpg
car :  352   /Volumes/alex_encrypted/model_v1/car/train2 / 001244 4.jpg
car :  353   /Volumes/alex_encrypted/model_v1/car/train2 / 000734 4.jpg
car :  354   /Volumes/alex_encrypted/model_v1/car/train2 / 000900 4.jpg
car :  355   /Volumes/alex_encrypted/model_v1/car/train2 / 000744 3.jpg
car :  356   /Volumes/alex_encrypted/model_v1/car/train2 / 000759 3.jpg
car :  357   /Volumes/alex_encrypted/model_v1/car/train2 / 001353 4.jpg
car :  358   /Volumes/alex_encrypted/model_v1/car/train2 / 000224.jpg
car :  359   /Volumes/alex_encrypted/model_v1/car/train2 / 001084 2.jpg
car :  360   /Volumes/alex_encrypted/model_v1/car/train2 / 001142 3.jp

car :  463   /Volumes/alex_encrypted/model_v1/car/train2 / 001160.jpg
car :  464   /Volumes/alex_encrypted/model_v1/car/train2 / 001202 3.jpg
car :  465   /Volumes/alex_encrypted/model_v1/car/train2 / 000382.jpg
car :  466   /Volumes/alex_encrypted/model_v1/car/train2 / 000077 3.jpg
car :  467   /Volumes/alex_encrypted/model_v1/car/train2 / 000204 4.jpg
car :  468   /Volumes/alex_encrypted/model_v1/car/train2 / 000735 3.jpg
car :  469   /Volumes/alex_encrypted/model_v1/car/train2 / 000083 2.jpg
car :  470   /Volumes/alex_encrypted/model_v1/car/train2 / 000448 4.jpg
car :  471   /Volumes/alex_encrypted/model_v1/car/train2 / 000380.jpg
car :  472   /Volumes/alex_encrypted/model_v1/car/train2 / 001300 3.jpg
car :  473   /Volumes/alex_encrypted/model_v1/car/train2 / 001246.jpg
car :  474   /Volumes/alex_encrypted/model_v1/car/train2 / 000057 3.jpg
car :  475   /Volumes/alex_encrypted/model_v1/car/train2 / 000144 4.jpg
car :  476   /Volumes/alex_encrypted/model_v1/car/train2 / 000499 3.jpg


...

odometer :  236   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000669.jpg
odometer :  237   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000664.jpg
odometer :  238   /Volumes/alex_encrypted/model_v1/odometer/train2 / 001271.jpg
odometer :  239   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000339.jpg
odometer :  240   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000637.jpg
odometer :  241   /Volumes/alex_encrypted/model_v1/odometer/train2 / 001278.jpg
odometer :  242   /Volumes/alex_encrypted/model_v1/odometer/train2 / 001052.jpg
odometer :  243   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000725.jpg
odometer :  244   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000058.jpg
odometer :  245   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000865.jpg
odometer :  246   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000670.jpg
odometer :  247   /Volumes/alex_encrypted/model_v1/odometer/train2 / 001259.jpg
odometer :  248   /Volumes/alex_encrypte

odometer :  339   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000524.jpg
odometer :  340   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000953.jpg
odometer :  341   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000023.jpg
odometer :  342   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000261.jpg
odometer :  343   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000054.jpg
odometer :  344   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000176.jpg
odometer :  345   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000883.jpg
odometer :  346   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000754.jpg
odometer :  347   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000939.jpg
odometer :  348   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000324.jpg
odometer :  349   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000187.jpg
odometer :  350   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000940.jpg
odometer :  351   /Volumes/alex_encrypte

odometer :  442   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000337.jpg
odometer :  443   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000810.jpg
odometer :  444   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000486.jpg
odometer :  445   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000185.jpg
odometer :  446   /Volumes/alex_encrypted/model_v1/odometer/train2 / 001397.jpg
odometer :  447   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000145.jpg
odometer :  448   /Volumes/alex_encrypted/model_v1/odometer/train2 / 001186.jpg
odometer :  449   /Volumes/alex_encrypted/model_v1/odometer/train2 / 001116.jpg
odometer :  450   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000083.jpg
odometer :  451   /Volumes/alex_encrypted/model_v1/odometer/train2 / 001084.jpg
odometer :  452   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000371.jpg
odometer :  453   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000427.jpg
odometer :  454   /Volumes/alex_encrypte

odometer :  545   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000582.jpg
odometer :  546   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000237.jpg
odometer :  547   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000635.jpg
odometer :  548   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000879.jpg
odometer :  549   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000897.jpg
odometer :  550   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000935.jpg
odometer :  551   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000447.jpg
odometer :  552   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000221.jpg
odometer :  553   /Volumes/alex_encrypted/model_v1/odometer/train2 / 001134.jpg
odometer :  554   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000538.jpg
odometer :  555   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000558.jpg
odometer :  556   /Volumes/alex_encrypted/model_v1/odometer/train2 / 000764.jpg
odometer :  557   /Volumes/alex_encrypte

OSError: cannot identify image file '/Volumes/alex_encrypted/model_v1/odometer/train/.DS_Store'

In [ ]:
checkpath(img_out_path)